In [ ]:
from siphon.catalog import TDSCatalog
import intake_esm
import xarray as xr
from pathlib import Path
from IPython.display import HTML, Markdown
import panel as pn
import hvplot
import hvplot.pandas
import hvplot.xarray
from bokeh.models.tools import HoverTool
import pandas as pd
pn.extension()
import warnings
warnings.simplefilter('ignore')

In [ ]:
intake_path = Path('intake_cats')
cats  = [l for l in list(intake_path.glob('*.json')) if l.name != 'cmip5.json']
cat = intake_esm.intake.open_esm_datastore(cats[-1])
cats

In [ ]:
## Climate simulations - bias adjusted
# bias adjusted
c = cats[3]
cat = intake_esm.intake.open_esm_datastore(c)
options = list(cat.df['title'].unique())
options1 = [o for o in options if 'Ouranos' in o]
options1.extend( [o for o in options if 'Ouranos' not in o])
title_w = pn.widgets.Select(options=options1, width = 600)
title_w

In [ ]:
@pn.depends(title_w.param.value)
def create_data_summary(dataset = title_w.param.value):
    df = None
    for c in cats:   
        cat = intake_esm.intake.open_esm_datastore(c)
        if len(cat.search(title=dataset).df) > 0:
            df = cat.search(title=dataset).df
            break
    ds = xr.open_dataset(df['path'][0], chunks=dict(time=1)).sortby(['lat','lon'])
    lic = ds.attrs['license']
    
    # bias corr specific info
    if 'driving_experiment' in df.columns:
        
        exp1 = [d.split(',') for d in df['driving_experiment'].unique()]
        exp1 = sorted(list({x for l in exp1 for x in l}))
    else:
        exp1 = None 
    if 'project_id' in df.columns and 'processing' in df.columns:
        prj1 = f"{df['project_id'].unique()[0]} ({df['processing'].unique()[0]})"
    else:
        prj1 = None
        

    w = 175

    ## summary info
    
    thrds_access = f"https://{'/'.join([p for p in df['path'][0].split('//')[-1].split('/')[0:-1]])}/catalog.html".replace('dodsC','catalog')
    thrds_xml = thrds_access.replace('.html','.xml')
    
    summary = pn.Column(pn.Row(pn.pane.HTML("dataset :", width=w), pn.pane.HTML(f'<a href="{thrds_access}" target="_blank">{dataset}<a />',width=600)))
    summary.append(pn.Row(pn.pane.HTML("thredds catalog :", width=w), pn.pane.HTML(f'<a href="{thrds_xml}" target="_blank">{thrds_xml}<a />',width=600)))
    summary.append(pn.Row(pn.pane.HTML("access tutorial :", width=w), pn.pane.HTML(f'<a href="/climate_analysis.html" target="_blank">PAVICS data tutorial<a />',width=600)))
    inst_field = 'institution' if 'insitution' in df.columns else 'institute'
    summary.append(pn.Row(pn.pane.HTML(f"{inst_field} :",width=w), pn.pane.HTML(df[inst_field].unique()[0], width=700)))
    if prj1:
        summary.append(pn.Row(pn.pane.HTML("project (processing level) :", width=w), pn.pane.HTML(prj1)))
    summary.append(pn.Row(pn.pane.HTML("frequency :", width=w), pn.pane.HTML(df['frequency'].unique()[0])))
    summary.append(pn.Row(pn.pane.HTML("variables :", width=w), pn.pane.HTML(', '.join([v for v in ds.data_vars]))))
    if exp1 :
        summary.append(pn.Row(pn.pane.HTML("driving experiment(s) :", width=w), pn.pane.HTML(', '.join(exp1))))

    out = pn.Tabs(('Summary',summary))

    ## details
    print(ds.attrs)
    details = pn.Column(pn.Row(pn.pane.HTML('abstract : ',width=w), pn.pane.HTML(ds.attrs['abstract'],width=600)))
    for check in ['bias_adjust', 'target_data', 'target_ref']:
        for attr in [attr for attr in ds.attrs if check in attr]:
            details.append(pn.Row(pn.pane.HTML(f"{attr.replace('_',' ')}: ", width=w), pn.pane.HTML(ds.attrs[attr],width=600 )))
    details.append(pn.Row(pn.pane.HTML('more info : ', width=w), pn.pane.HTML(ds.attrs['dataset_description'], width=600)))
    out.append(('Details', details))

    ## legal
    legal = pn.Column(pn.Row(pn.pane.HTML("license type :", width=w), pn.pane.HTML(ds.attrs['license_type'], width=600)))
    legal.append(pn.Row(pn.pane.HTML("license :", width=w), pn.pane.HTML(ds.attrs['license'],width=600)))
    
    for check in ['attribution', 'citation', 'terms']:
        for attr in [attr for attr in ds.attrs if check in attr]:
            legal.append(pn.Row(pn.pane.HTML(f"{attr.replace('_',' ')}: ", width=w), pn.pane.HTML(ds.attrs[attr],width=600 )))
            
    out.append(('License / Terms of use', legal))

    ## map
    v = list(ds.data_vars.keys())
    map1 = ds[v[0]].isel(time=0).hvplot.image(cmap='RdBu_r',datashade=True, hover=False, frame_height=300,frame_width=700)

    out1 = pn.Column(map1,out)
    return out1

In [ ]:
MAX_WIDTH=850

pn.config.sizing_mode="stretch_width"

spacer = pn.Spacer(height=0, margin=0)
main_content =  pn.Column(title_w,create_data_summary,sizing_mode="stretch_width", 
    max_width=MAX_WIDTH, align="center" )

main_area = pn.Column(
    spacer, # TRICK: WONT WORK WITHOUT. YOU CAN SET HEIGHT TO 0 TO NOT TAKE UP HEIGHT
    main_content,
    sizing_mode="stretch_both",
    
)

#main_area.servable()
#from bokeh.resources import INLINE
#main_area.save('Datasets_1-Climate_Simulations.html', embed=True)

In [ ]:
pn.extension(loading_spinner='bars', loading_color='#00aa41')
#main_area.servable()

In [47]:
options1 = []
for c in [c for c in cats if 'obs.json' in c.name]:

    cat = intake_esm.intake.open_esm_datastore(c)
    options1.extend( list(cat.df['title'].unique()))
#options1.extend( [o for o in options if 'Ouranos' not in o])
title_w = pn.widgets.Select(options=options1, width = 600)
title_w
MAX_WIDTH=850

pn.config.sizing_mode="stretch_width"

spacer = pn.Spacer(height=0, margin=0)
observed =  pn.Column(title_w,create_data_summary,sizing_mode="stretch_width", 
    max_width=MAX_WIDTH, align="center" )

observed = pn.Column(
    spacer, # TRICK: WONT WORK WITHOUT. YOU CAN SET HEIGHT TO 0 TO NOT TAKE UP HEIGHT
    observed,
    sizing_mode="stretch_both",
    
)
observed

{'Conventions': 'CF-1.5', 'title': 'NRCAN ANUSPLIN daily gridded dataset : version 2', 'history': 'Fri Jan 25 14:11:15 2019 : Convert from original format to NetCDF', 'institute_id': 'NRCAN', 'frequency': 'day', 'type': 'gridded_obs', 'institute': 'Natural Resources Canada : Canadian Forest Service', 'dataset_id': 'NRCAN_anusplin_daily_v2', 'version': '2.0', 'license_type': 'permissive', 'license': 'https://open.canada.ca/en/open-government-licence-canada', 'attribution': 'The authors provide this data under the Environment and Climate Change Canada Data Server End Use License and demand that any references to or derivative works based on this dataset be cited using the provided citation information', 'citation': 'Natural Resources Canada ANUSPLIN interpolated historical climate model from McKenney, D.W., M.F. Hutchinson, P. Papadol, K. Lawrence, J. Pedlar, K. Campbell, E. Milewska, R.F. Hopkinson, D. Price, and T. Owen, 2011. Customized Spatial Climate Models for North America. Bull. 

KeyError: 'abstract'

In [ ]:
cats
